In [301]:
#!/usr/bin/env python3
"""
Read SO-CI output from Molpro
Assign omegas, including parity
Also show composition of a level, or distribution of a term
MRCI+Q diagonal energies are assumed
KKI March 2023: include dipole moment

Use "gexpec,LZ" at the top of input and "option,matel=1" in the SO-CI command
"""
import re, sys, copy, glob, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from termcolor import colored
from collections import Counter

sys.path.insert(0, '../atomic_SOC')
#import molpro_subs as mpr
import chem_subs as chem
import molpro_subs2 as m2
import assign_PtO as pto

pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 200)

In [302]:
molec = 'PtO'
read_dipoles = True  #  whether to read dipole moments

In [783]:
if molec == 'PtO':
    # neutral PtO
    fdir = r'C:\Users\irikura\OneDrive - NIST\Karl\PtO\14S_17T_5Q\\'
    froot = 'ac5z_r1p727_lz.pro'
    #fdir = r'C:\Users\irikura\OneDrive - NIST\Karl\PtO\Molpro_test\\'
    #froot = 'actz_13S15T_X2C_b.pro'
    #fdir = r'C:\Users\irikura\OneDrive - NIST\Karl\PtO\Molpro_test\O_test\\'
    #froot = 'actzppo_14S17T.pro'
else:
    print(f'Unhandled molecule {molec}')
    print('Add values of "fdir" and "froot"')

In [784]:
fsoci = fdir + froot
print('Will read SO-CI info for {:s} from file: {:s}'.format(molec, froot))

Will read SO-CI info for PtO from file: ac5z_r1p727_lz.pro


In [785]:
# Identify the different sections of the output file
major_sections, linenos = pto.identify_major_sections(fsoci)
allECP = ('SOintegrals' not in major_sections.keys())

Major sections:
   input         1 text blocks
   header        1 text blocks
   integrals     1 text blocks
   rhf           1 text blocks
   multi         1 text blocks
   mrci          17 text blocks
   SOintegrals   1 text blocks
   soci          1 text blocks


In [786]:
# section "input" is ignored

In [787]:
# section "header"
basisset = m2.basisset_name(major_sections['header'][0])
print(basisset)

AUG-CC-PWCV5Z-PP,O=AUG-CC-PWCV5Z


In [788]:
PG, nprim, nbf, crd, Qtot, Rbohr, R, Rstr = \
    pto.parse_section_integrals(major_sections['integrals'][0])

Point group = C2v
There are 436 contracted basis functions from 722 primitives
Atoms:
   {'nr': 1, 'el': 'Pt', 'Qnuc': 18.0, 'xyzBohr': [0.0, 0.0, -0.247371151]}
   {'nr': 2, 'el': 'O', 'Qnuc': 8.0, 'xyzBohr': [0.0, 0.0, 3.016185866]}
Total nuclear charge = 26
R = 1.727 Å


In [789]:
occup_hf, nel_HF, hf_results, orbtitle, dfHForb = \
    pto.parse_section_rhf(major_sections['rhf'][0], Qtot)

RHF occupations:  {'alpha': [7, 3, 3, 1], 'beta ': [7, 2, 2, 1]}
HF has 26 electrons (charge = 0)
RHF energy = -193.315552 for state 1.4
ELECTRON ORBITALS


Orbital,Occ,Energy,Cen,Mu,Typ,Coeff
1.1,2.000000,-20.649050,2,1,s,1.00086
2.1,2.000000,-4.444700,1,1,s,0.99861
3.1,2.000000,-2.625390,1,1,pz,0.98844
4.1,2.000000,-1.277550,2,2,s,0.95369
5.1,2.000000,-0.627360,1,1,d0,-0.46754
5.1,2.000000,-0.627360,2,1,pz,0.81221
6.1,2.000000,-0.432420,1,1,d2+,0.99823
7.1,2.000000,-0.344750,1,2,s,-0.69852
7.1,2.000000,-0.344750,1,1,d0,0.66316
7.1,2.000000,-0.344750,2,1,pz,0.29837


### Parse CASSCF

In [790]:
# break last section "multi" into sub-sections
multisec = m2.multi_sections(major_sections['multi'][-1])
multisec.keys()

dict_keys(['top', 'iterations', 'results', 'trans', 'natorb', 'civector', 'tail'])

In [791]:
dynfac, orbspace, statesym, convergence, weights, weights_ini,\
    weightlbls, dfiter, dfstates, N_cas, dfexpec, dftrans, orbtitle,\
    dfNO, ddfcivec, dEcas, nactel, nactorb, multip_count = \
    pto.parse_MULTI(multisec)

36 CASSCF states
DYNW = 8.0
CASSCF(14/9)
Spaces:  {'closed-shell': [4, 1, 1, 0], 'active': [4, 2, 2, 1], 'external': [148, 103, 103, 67]}
State counts:  {'Singlet': 14, 'Triplet': 17, 'Quintet': 5}
NATURAL ORBITALS (STATE AVERAGED)


Orbital,Occ,Energy,Cen,Mu,Typ,Coeff
1.1,2.000000,-20.597330,2,1,s,1.00076
2.1,2.000000,-4.498660,1,1,s,0.99865
3.1,2.000000,-2.667920,1,1,pz,0.98959
4.1,2.000000,-1.258220,2,2,s,0.95874
5.1,1.937650,-0.593660,1,1,d0,-0.57276
5.1,1.937650,-0.593660,2,1,pz,0.73238
6.1,1.798120,-0.438210,1,1,d2+,0.99799
7.1,1.589690,-0.298440,1,2,s,-0.79184
7.1,1.589690,-0.298440,1,1,d0,0.55765
7.1,1.589690,-0.298440,2,1,pz,0.27542


None

### Gather CASSCF information in a DataFrame

In [792]:
dfcas, dfsmult_emax = pto.build_CASSCF_dataframe(dfstates, dfexpec, statesym,\
                                multip_count,dynfac, weights)

Highest state of each spin multiplicity (cm-1):


,CASSCF
Singlet,26837.0
Triplet,26602.5
Quintet,28068.1


Dynamical weights used in state-averaged CASSCF
Largest deviation from integer LZLZ is -1.0e-12


In [793]:
# Build DataFrame of CASSCF terms
dftermcas, ncas, nterm = pto.build_CASSCF_term_DF(dfcas)

Largest accepted energy non-degeneracy = 0.0 cm-1 for [20 24]
CASSCF: 22 terms from 36 states


In [794]:
dftermcounts = pto.CASSCF_term_counts(dftermcas, multip_count)
dftermcas.sort_values(['S', 'Lz', 'Term'])
dftermcas.sort_values('Ecm')

Term counts from CASSCF


,Σ+,Σ-,Π,Δ,Φ
Singlet,2,2,3,1,1
Triplet,1,2,3,3,1
Quintet,0,1,0,1,1


,Term,index,Label,E,dipZ,irrep,LZLZ,spinM,S,CASwt,Lz,Ecm
5,3Σ-,[26],[1.4 Triplet],-193.398871,-1.126405,[4],[-0.0],Triplet,1.0,[1.0],0.0,0.0
15,1Δ,"[2, 13]","[3.1 Singlet, 3.4 Singlet]",-193.375940,-1.123668,"[1, 4]","[3.999999999999, 3.999999999999]",Singlet,0.0,"[0.96709, 0.96709]",2.0,5032.6
11,3Π,"[18, 22]","[1.2 Triplet, 1.3 Triplet]",-193.368478,-1.237833,"[2, 3]","[1.0, 1.0]",Triplet,1.0,"[0.943136, 0.943136]",1.0,6670.5
8,1Π,"[3, 7]","[1.2 Singlet, 1.3 Singlet]",-193.359147,-1.218019,"[2, 3]","[1.0, 1.0]",Singlet,0.0,"[0.905438, 0.905438]",1.0,8718.4
1,1Σ+,[0],[1.1 Singlet],-193.359009,-1.119345,[1],[-0.0],Singlet,0.0,[0.904825],0.0,8748.6
0,5Σ-,[34],[1.4 Quintet],-193.349160,-0.357075,[4],[-0.0],Quintet,2.0,[0.857142],0.0,10910.2
21,3Φ,"[21, 25]","[4.2 Triplet, 4.3 Triplet]",-193.347831,-1.077085,"[2, 3]","[8.999999999999, 8.999999999999]",Triplet,1.0,"[0.850195, 0.850195]",3.0,11202.0
12,3Π,"[19, 23]","[2.2 Triplet, 2.3 Triplet]",-193.342118,-0.983804,"[2, 3]","[1.0, 1.0]",Triplet,1.0,"[0.819206, 0.819206]",1.0,12455.8
9,1Π,"[4, 8]","[2.2 Singlet, 2.3 Singlet]",-193.329985,-1.029579,"[2, 3]","[1.0, 1.0]",Singlet,0.0,"[0.748663, 0.748663]",1.0,15118.7
20,1Φ,"[6, 10]","[4.2 Singlet, 4.3 Singlet]",-193.327642,-1.016900,"[2, 3]","[9.0, 9.0]",Singlet,0.0,"[0.734531, 0.734531]",3.0,15633.0


### Demonstrate parsing one MRCI (optional)

In [795]:
imrci = None
pto.demonstrate_MRCI_parsing(major_sections['mrci'], imrci)  # does nothing if imrci is None

### Gather MRCI info in a DataFrame

In [796]:
dfmrci, nstate = pto.build_MRCIs_DF(major_sections['mrci'], ncas)

MRCI #0   :   2 states   irrep=1   Singlet, 5101.2
MRCI #1   :   1 states   irrep=1   Singlet, 5102.2
MRCI #2   :   4 states   irrep=2   Singlet, 5201.2
MRCI #3   :   4 states   irrep=3   Singlet, 5301.2
MRCI #4   :   2 states   irrep=4   Singlet, 5401.2
MRCI #5   :   1 states   irrep=4   Singlet, 5402.2
MRCI #6   :   1 states   irrep=1   Triplet, 6101.2
MRCI #7   :   3 states   irrep=1   Triplet, 6102.2
MRCI #8   :   4 states   irrep=2   Triplet, 6201.2
MRCI #9   :   4 states   irrep=3   Triplet, 6301.2
MRCI #10  :   2 states   irrep=4   Triplet, 6401.2
MRCI #11  :   3 states   irrep=4   Triplet, 6402.2
MRCI #12  :   1 states   irrep=1   Quintet, 7101.2
MRCI #13  :   1 states   irrep=2   Quintet, 7201.2
MRCI #14  :   1 states   irrep=3   Quintet, 7301.2
MRCI #15  :   1 states   irrep=4   Quintet, 7401.2
MRCI #16  :   1 states   irrep=4   Quintet, 7402.2


In [797]:
# Report the highest energy state of each spin mult (relative cm-1)
emin = dfmrci.Edav.min()
erel = [(dfmrci[dfmrci.spinM == smult]['Edav'].max() - emin) * chem.AU2CM \
                         for smult in multip_count.keys()]
dfsmult_emax['MRCI+Q'] = np.round(erel, 1)
print('Highest state of each spin multiplicity (cm-1):')
dfsmult_emax

Highest state of each spin multiplicity (cm-1):


,CASSCF,MRCI+Q
Singlet,26837.0,28467.3
Triplet,26602.5,28855.5
Quintet,28068.1,32244.8


In [798]:
# Match MRCI states with their CASSCF states
#    A failure to match means that there is an intruder state in MRCI
nfail = pto.match_MRCI_to_CASSCF(dfcas, dfmrci, 1.e-6)

Matching 36 MRCI states to 36 CASSCF states (etol = 1e-06)...successful.


In [799]:
# Examine leading configurations to check matching between CASSCF and MRCI states
if nfail:
    print('-*-*-*- The CASSCF/MRCI state-matching failed -*-*-*-')
    1/0
else:
    pto.compare_CAS_MRCI_configurations(dfcas, dfmrci, ddfcivec, major_sections['mrci'])

Checking leading configurations to verify MRCI/CASSCF matching

MRCI state 0 (cigroup 0, S=0, label=2.1) has CASSCF ref state 0
    leading configs same
MRCI state 1 (cigroup 0, S=0, label=3.1) has CASSCF ref state 1
    leading configs same
MRCI state 2 (cigroup 1, S=0, label=1.1) has CASSCF ref state 2
    leading configs same
MRCI state 3 (cigroup 2, S=0, label=1.2) has CASSCF ref state 3
    MRCI set is subset of CASSCF set
MRCI state 4 (cigroup 2, S=0, label=2.2) has CASSCF ref state 4
    MRCI set is subset of CASSCF set
MRCI state 5 (cigroup 2, S=0, label=3.2) has CASSCF ref state 6
    MRCI set is subset of CASSCF set
MRCI state 6 (cigroup 2, S=0, label=4.2) has CASSCF ref state 5
    MRCI set is subset of CASSCF set
MRCI state 7 (cigroup 3, S=0, label=1.3) has CASSCF ref state 7
    MRCI set is subset of CASSCF set
MRCI state 8 (cigroup 3, S=0, label=2.3) has CASSCF ref state 8
    MRCI set is subset of CASSCF set
MRCI state 9 (cigroup 3, S=0, label=3.3) has CASSCF ref state 1

,irr 1,irr 2,irr 3,irr 4,1,csq
1,22//,2\,2/,2,0.713373,0.508902
2,22//,2/,2\,2,0.504431,0.254451
3,22/\,2/,2/,2,0.212412,0.045119
5,22//,\2,/2,2,-0.134015,0.017960
10,22//,/2,\2,2,-0.094763,0.008980


    CASSCF configurations (leading MRCI config highlighted):


,irr 1,irr 2,irr 3,irr 4,state 2,csq
4,22ba,2a,2a,2,0.604608,0.365551
2,22aa,2a,2b,2,-0.455758,0.207715
3,22aa,2b,2a,2,-0.455758,0.207715
8,22ab,2a,2a,2,0.306907,0.094192
18,22ba,a2,a2,2,-0.140770,0.019816


    MRCI state 27:


,spinM,S,Label,cigroup,E,Edav,Eref,ref0,C0,dipZ,irrep,saverec,refchange,iref,ref0dif
27,Triplet,1,3.4,10,-193.838191,-193.90141,-193.310847,-193.310782,0.94496,-0.205532,4,6401.2,-0.000064,27,-6.340031e-10


    CASSCF state 27:


,Label,E,dipZ,irrep,LZLZ,spinM,S,CASwt,Lz
27,2.4 Triplet,-193.310782,-0.323941,4,-0.0,Triplet,1,0.631129,0


MRCI state 28 (cigroup 11, S=1, label=2.4) has CASSCF ref state 28
    MRCI set is subset of CASSCF set
MRCI state 29 (cigroup 11, S=1, label=4.4) has CASSCF ref state 29
    leading configs same
MRCI state 30 (cigroup 11, S=1, label=5.4) has CASSCF ref state 30
    leading configs same
MRCI state 31 (cigroup 12, S=2, label=1.1) has CASSCF ref state 31
    leading configs same
MRCI state 32 (cigroup 13, S=2, label=1.2) has CASSCF ref state 32
    leading configs same
MRCI state 33 (cigroup 14, S=2, label=1.3) has CASSCF ref state 33
    leading configs same
MRCI state 34 (cigroup 15, S=2, label=1.4) has CASSCF ref state 34
    leading configs same
MRCI state 35 (cigroup 16, S=2, label=2.4) has CASSCF ref state 35
    leading configs same


In [800]:
if False:
    config = '22aa 2b 2a 2'  # active orbitals
    retd = pto.CASSCF_with_config(ddfcivec, config)

In [801]:
## Copy Lz from matched CASSCF
lz = [dfcas.at[i, 'Lz'] for i in dfmrci.iref]
dfmrci['Lz'] = lz
dfmrci[['E', 'Edav', 'ref0', 'irrep', 'S', 'Lz']].sort_values('Edav')

,E,Edav,ref0,irrep,S,Lz
26,-193.933121,-193.997608,-193.398871,4,1,0
13,-193.909854,-193.971805,-193.375940,4,0,2
2,-193.909854,-193.971805,-193.375940,1,0,2
22,-193.901863,-193.965546,-193.368478,3,1,1
18,-193.901863,-193.965546,-193.368478,2,1,1
7,-193.895079,-193.959455,-193.359147,3,0,1
3,-193.895079,-193.959455,-193.359147,2,0,1
0,-193.893734,-193.957175,-193.359009,1,0,0
23,-193.876523,-193.939668,-193.347831,3,1,3
19,-193.876523,-193.939668,-193.347831,2,1,3


In [802]:
# Build DataFrame of MRCI terms
dftermci = pto.build_MRCI_term_DF(dfmrci, nterm, ncas)
dftermci[['Term', 'Ecm', 'S', 'Lz', 'irrep', 'index']].sort_values('Ecm')

Largest accepted energy non-degeneracy = 0.0 cm-1 for [17 30]
MRCI: 22 terms from 36 states


,Term,Ecm,S,Lz,irrep,index
5,(1)3Σ-,0.0,1.0,0,[4],[26]
15,1Δ,5663.2,0.0,2,"[1, 4]","[2, 13]"
11,(1)3Π,7036.8,1.0,1,"[3, 2]","[22, 18]"
8,(1)1Π,8373.6,0.0,1,"[2, 3]","[3, 7]"
1,(1)1Σ+,8874.0,0.0,0,[1],[0]
21,3Φ,12716.4,1.0,3,"[3, 2]","[23, 19]"
12,(2)3Π,13784.9,1.0,1,"[2, 3]","[20, 24]"
0,5Σ-,14715.8,2.0,0,[4],[34]
9,(2)1Π,16248.1,0.0,1,"[2, 3]","[4, 8]"
20,1Φ,16925.4,0.0,3,"[2, 3]","[5, 9]"


In [803]:
# Report differences between initial and final reference energies
dfref = dftermci[['index', 'S', 'Term', 'Label', 'Ecm', 'Edav', 'Eref', 'ref0', 'iref']].copy()
dfref['ref_diff'] = np.round(dfref.Eref - dfref.ref0, 6)
print('Energy difference between "Eref" and "ref0" is "ref_diff":')
imax = np.abs(dfref['ref_diff']).idxmax()
print(f'Largest difference = {dfref.ref_diff.iloc[imax]}')
dfref.sort_values(['S', 'Edav'])
#dfref.sort_values('Edav')

Energy difference between "Eref" and "ref0" is "ref_diff":
Largest difference = 0.000359


,index,S,Term,Label,Ecm,Edav,Eref,ref0,iref,ref_diff
15,"[2, 13]",0.0,1Δ,"[1.1, 1.4]",5663.2,-193.971805,-193.375940,-193.375940,"[2, 13]",0.000000
8,"[3, 7]",0.0,(1)1Π,"[1.2, 1.3]",8373.6,-193.959455,-193.358984,-193.359147,"[3, 7]",0.000163
1,[0],0.0,(1)1Σ+,[2.1],8874.0,-193.957175,-193.359008,-193.359009,[0],0.000001
9,"[4, 8]",0.0,(2)1Π,"[2.2, 2.3]",16248.1,-193.923577,-193.330120,-193.329985,"[4, 8]",-0.000135
20,"[5, 9]",0.0,1Φ,"[3.2, 3.3]",16925.4,-193.920490,-193.327644,-193.327641,"[6, 10]",-0.000002
2,[1],0.0,(2)1Σ+,[3.1],20729.3,-193.903159,-193.276593,-193.276592,[1],-0.000001
10,"[6, 10]",0.0,(3)1Π,"[4.2, 4.3]",24862.7,-193.884325,-193.287395,-193.287370,"[5, 9]",-0.000025
3,[11],0.0,(1)1Σ-,[2.4],25463.2,-193.881589,-193.287642,-193.287701,[11],0.000060
4,[12],0.0,(2)1Σ-,[3.4],28467.3,-193.867902,-193.276934,-193.276874,[12],-0.000060
5,[26],1.0,(1)3Σ-,[1.4],0.0,-193.997608,-193.398806,-193.398871,[26],0.000064


### Parse SO-CI

In [804]:
# section "SOintegrals" 
if allECP:
    SOintgrl = None
else:
    SOintgrl = m2.SO_integrals(major_sections['SOintegrals'][0])
SOintgrl

{'integral_cutoff': 1e-10, 'SO_op_irreps': {'LSX': 3, 'LSY': 2, 'LSZ': 4}}

In [805]:
# break section "soci" into sub-sections
sosec = m2.soci_sections(major_sections['soci'][0])
sosec.keys()

dict_keys(['matel_comput', 'basis_prop', 'so_calc', 'so_levels', 'so_vectors', 'so_compos', 'prop', 'tail'])

In [806]:
if False:
    k = 'prop'
    iblock = 0
    print(len(sosec[k]), len(sosec[k][iblock]))
    for line in sosec[k][iblock]:
        print(line.rstrip()[:80])

In [807]:
# SOCI sub-section 'matel_comput'
hlsdiag, n_cistates, mat_elems = pto.check_SOCI_matelems(sosec['matel_comput'][0], dfmrci)

There are 36 MRCI states in the SO-CI


In [808]:
# SOCI sub-section 'basis_prop'
basprop = pto.SOCI_basprop(sosec['basis_prop'][0], n_cistates, dfmrci)
if 'LZ' not in basprop.keys():
    chem.print_err('', 'Must calculate LZ matrix elements for basis')
basprop.keys()

dict_keys(['DMX', 'DMY', 'DMZ', 'LZ', 'basis'])

In [809]:
# Was the LZ matrix computed?
know_LZ = ('LZ' in basprop.keys()) and (basprop['LZ'] is not None)
if know_LZ:
    print('LZ matrix will be used to characterize basis states')
else:
    print('Irreps will be used to characterize basis states')

LZ matrix will be used to characterize basis states


In [810]:
# SOCI sub-section 'so_calc'
E0 = m2.soci_E0(sosec['so_calc'][0])
print(f'E0 = {E0:.6f}')
if E0 > -0.5:
    print('   *** parsing failure for E0 ***')
    1/0
somat = m2.soci_matrix(sosec['so_calc'][0])
dimen = somat['matrix'].shape[0]
print(f'There are {dimen} SO-CI states')

E0 = -193.997608
There are 90 SO-CI states


In [811]:
# SOCI sub-section 'so_levels'
so_energies = m2.soci_energies(sosec['so_levels'][-1])
df_soE = pd.DataFrame(so_energies)
df_soE.sort_values('Erel')

,E,Eshift,Erel
0,-194.010209,-2765.51,0.00
1,-194.006312,-1910.20,855.31
2,-194.006312,-1910.18,855.33
3,-193.988327,2037.11,4802.62
4,-193.988317,2039.23,4804.74
5,-193.970860,5870.64,8636.15
6,-193.969326,6207.24,8972.75
7,-193.969326,6207.37,8972.88
8,-193.968434,6403.12,9168.63
9,-193.968429,6404.08,9169.59


In [812]:
# SO-CI sub-section 'so_vectors'
so_vecs = pto.SOCI_eigvecs(sosec['so_vectors'][-1], dimen)
so_vecs.keys()

dict_keys(['basis', 'matrix'])

In [813]:
# SO-CI sub-section 'so_compos'
try:
    so_compos = m2.soci_composition(sosec['so_compos'][0])
except:
    print('Using special parser')
    so_compos = m2.soci_composition_even(sosec['so_compos'][0])
so_compos.keys()

dict_keys(['basis', 'matrix'])

In [814]:
# check that all orderings of basis states are consistent
basord_ok, basord = pto.SOCI_basord_check(somat, so_vecs, so_compos)

Basis states are consistent between eigenvectors and composition
Basis state are also consistent between matrix and eigenvectors


In [815]:
# Check that eigenvectors are consistent with printed composition, and
#    that the printed eigenvalues are consistent with the SO-CI matrix
difmat, dmax, my_eigvals, my_eigvecs, my_ok = \
    pto.SOCI_eig_check(so_vecs['matrix'], so_compos['matrix'], somat['matrix'], df_soE)

Largest inconsistency between printed composition and printed eigenvectors = 0.000%
Largest difference between printed and re-computed eigenvalues is -0.01 cm-1


In [816]:
use_recalculated = True  # use re-calculated eigenvectors (else use printed eigenvectors)

In [817]:
df_soE[chem.OMEGA] = np.inf   # value for unassigned levels
if use_recalculated:
    print('Using eigenvectors determined directly from the SO-CI matrix')
    eigenvectors = my_eigvecs
    compositions = np.conjugate(my_eigvecs) * my_eigvecs
    compositions = np.real(compositions) * 100.  # convert to percent
else:
    print('Using eigenvectors and compositions as printed in the Molpro file')
    eigenvectors = so_vecs['matrix']
    compositions = so_compos['matrix']
magnit = np.conjugate(eigenvectors) * eigenvectors
magnit = np.real(magnit)

Using eigenvectors determined directly from the SO-CI matrix


In [818]:
# SO-CI sub-section 'prop'
so_propmats = pto.SOCI_property_mats(sosec['prop'][0], dimen)

Property matrices for final states:
    ['DMX_REAL', 'DMX_IMAG', 'DMY_REAL', 'DMY_IMAG', 'DMZ_REAL', 'DMZ_IMAG', 'LZ_REAL', 'LZ_IMAG', 'vector', 'magn', 'LZ']


In [819]:
# SO-CI sub-section 'prop' (continued)
so_expec = m2.soci_expec(sosec['prop'][0], dimen)
print('Expectation values listed individually: ', list(so_expec.keys()))
so_trans = m2.soci_trans(sosec['prop'][0], dimen)

*** No values for <i|DMZ|i> for state = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]
*** No values for <i|LZ|i> for state = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]
Expectation values listed individually:  ['DMX', 'DMY', 'DMZ', 'LZ', 'print_thr']
*** No values for <i|DMX|1> for bra = [73, 74, 75, 76, 77, 78, 79, 80]
*** No values for <i|DMZ|1> for bra = [65, 66, 67, 68, 69, 70, 7

In [820]:
tCPU, tWALL, wallt = m2.final_times(fsoci)

The calculation took 6.9 hours


### Characterize SO-CI basis states

In [821]:
dfsobas = pto.match_somat_MRCI_labels(somat, E0, dfmrci)

All basis states matched to MRCI states based on irrep, S, and energy
Basis states in the SO-CI matrix:


,Nr,State,S,Sz,Ecm,E,i_ci,ci_lbl,Lz,irrep,saverec,Ω,uOm
0,1,1.1,0.0,0.0,8874.05,-193.957175,0,2.1,0,1,5101.2,{0.0},{0.0}
1,2,2.1,0.0,0.0,20729.29,-193.903159,1,3.1,0,1,5101.2,{0.0},{0.0}
2,3,1.1,0.0,0.0,5663.24,-193.971805,2,1.1,2,1,5102.2,"{2.0, -2.0}",{2.0}
3,4,1.2,0.0,0.0,8373.64,-193.959455,3,1.2,1,2,5201.2,"{1.0, -1.0}",{1.0}
4,5,2.2,0.0,0.0,16248.10,-193.923577,4,2.2,1,2,5201.2,"{1.0, -1.0}",{1.0}
5,6,3.2,0.0,0.0,16925.42,-193.920491,5,3.2,3,2,5201.2,"{3.0, -3.0}",{3.0}
6,7,4.2,0.0,0.0,24862.71,-193.884326,6,4.2,1,2,5201.2,"{1.0, -1.0}",{1.0}
7,8,1.3,0.0,0.0,8373.62,-193.959455,7,1.3,1,3,5301.2,"{1.0, -1.0}",{1.0}
8,9,2.3,0.0,0.0,16248.13,-193.923576,8,2.3,1,3,5301.2,"{1.0, -1.0}",{1.0}
9,10,3.3,0.0,0.0,16925.42,-193.920491,9,3.3,3,3,5301.2,"{3.0, -3.0}",{3.0}


In [822]:
# Assign basis states to pairs
sobaspair = pto.pair_basis_states_C2v(dfsobas, dimen, dftermci, verbose=False)

In [823]:
# Assign sin/cos character to basis states
if know_LZ:
    print('Assigning sin/cos character to basis states using using <LZ> matrix')
    maxerr = pto.assign_trig_by_LZmat(dfsobas, basprop['LZ'], basprop['basis'], maxerr=0.12)
else:
    print('Assigning sin/cos character to basis states using using irreps')
    pto.assign_trig_by_irrep(dfsobas)
used_LZ = know_LZ
dfail = dfsobas[dfsobas.type == '?']
if len(dfail):
    print('*** Failed to assign trig character of some basis states ***')
    display(dfail)
    q = input('Accept them all, anyway? ')
    if (len(q) > 0) and (q.lower()[0] == 'y'):
        pto.assign_trig_by_LZmat(dfsobas, basprop['LZ'], basprop['basis'], maxerr=abs(maxerr))
        dfail = dfsobas[dfsobas.type == '?']
        if len(dfail):
            display(dfail)

Assigning sin/cos character to basis states using using <LZ> matrix
*** lz =   1 but elem = -0.99986, err = -0.00014
*** lz =   1 but elem = 0.99618, err = -0.00382
*** lz =   3 but elem = 2.99594, err = -0.00406
*** lz =   1 but elem = -0.99986, err = -0.00014
*** lz =   3 but elem = -2.99758, err = -0.00242
*** lz =   1 but elem = -0.99776, err = -0.00224
*** lz =   1 but elem = -0.99989, err = -0.00011
*** lz =   3 but elem = -2.99758, err = -0.00242
*** lz =   1 but elem = -0.99776, err = -0.00224
*** lz =   1 but elem = -0.99989, err = -0.00011
*** lz =   3 but elem = -2.99758, err = -0.00242
*** lz =   1 but elem = -0.99776, err = -0.00224
*** lz =   1 but elem = -0.99989, err = -0.00011


### Find term compositions of sublevels

In [824]:
dfcompos, dfcompostot = pto.term_compositions_of_levels(df_soE, dftermci, dfsobas, 
                                                       magnit*100)
display(dfcompostot.style.format('{:.2f}'))

Total term compositions (%):


,%,err
5Σ-,500.00,-0.00
(1)1Σ+,100.00,0.00
(2)1Σ+,100.00,0.00
(1)1Σ-,100.00,0.00
(2)1Σ-,100.00,0.00
(1)3Σ-,300.00,-0.00
(2)3Σ-,300.00,-0.00
3Σ+,300.00,0.00
(1)1Π,200.00,-0.00
(2)1Π,200.00,-0.00


# Assign Ω and parity

In [825]:
# Determine the values of Omega required by the input MRCI states
omcount, omsublev = pto.omegas_from_terms(dftermci, dimen)

Required Ω counts (levels):
    {0: 16, 1: 15, 2: 11, 3: 7, 4: 3, 5: 1}
Required Ω counts (sublevels):
    {0: 16, 1: 30, 2: 22, 3: 14, 4: 6, 5: 2}


In [826]:
# bs_coeff[iso] is a DataFrame of significant basis-state contributions
# trig_rept[iso] shows trig analysis for contributing basis states
# om_compl[iso] is a dict summarizing Omega contributions, with weights
bs_coeff, trig_rept, omcompl = pto.SOCI_eigvec_trig_analysis(dfsobas, df_soE, 
                                    eigenvectors, compositions, dimen, thr=0.02, verbose=True)

Examining SO-CI states one at a time
Composition threshold = 0.02% to consider a basis state

--- iso = 0 with Erel = 0.0 cm-1 ---


,Nr,State,S,Sz,Ecm,E,i_ci,ci_lbl,Lz,irrep,saverec,Ω,uOm,ipair,Term,type,pct,coeff
52,53,1.4,1.0,0.0,0.00,-193.997608,26,1.4,0,4,6401.2,{0.0},{0.0},52,(1)3Σ-,1,80.107468,-0.000+0.895j
0,1,1.1,0.0,0.0,8874.05,-193.957175,0,2.1,0,1,5101.2,{0.0},{0.0},0,(1)1Σ+,1,8.921185,0.299+0.000j
40,41,3.3,1.0,1.0,13784.91,-193.934800,24,3.3,1,3,6301.2,"{0.0, 2.0}","{0.0, 2.0}",28,(2)3Π,sin,1.325293,0.000+0.115j
48,49,3.3,1.0,-1.0,13784.91,-193.934800,24,3.3,1,3,6301.2,"{0.0, -2.0}","{0.0, 2.0}",36,(2)3Π,sin,1.325293,0.000-0.115j
28,29,3.2,1.0,1.0,13784.89,-193.934800,20,3.2,1,2,6201.2,"{0.0, 2.0}","{0.0, 2.0}",40,(2)3Π,cos,1.325196,-0.115+0.000j
36,37,3.2,1.0,-1.0,13784.89,-193.934800,20,3.2,1,2,6201.2,"{0.0, -2.0}","{0.0, 2.0}",48,(2)3Π,cos,1.325196,-0.115+0.000j
26,27,1.2,1.0,1.0,7036.86,-193.965546,18,1.2,1,2,6201.2,"{0.0, 2.0}","{0.0, 2.0}",38,(1)3Π,cos,1.047520,0.102+0.000j
34,35,1.2,1.0,-1.0,7036.86,-193.965546,18,1.2,1,2,6201.2,"{0.0, -2.0}","{0.0, 2.0}",46,(1)3Π,cos,1.047520,0.102+0.000j
38,39,1.3,1.0,1.0,7036.83,-193.965546,22,1.3,1,3,6301.2,"{0.0, 2.0}","{0.0, 2.0}",26,(1)3Π,sin,1.046928,0.000-0.102j
46,47,1.3,1.0,-1.0,7036.83,-193.965546,22,1.3,1,3,6301.2,"{0.0, -2.0}","{0.0, 2.0}",34,(1)3Π,sin,1.046928,0.000+0.102j


non-pair component 52 (Λ=0) is Ω = 0.0
non-pair component 0 (Λ=0) is Ω = 0.0
pair combination [28, 40] with c=-0.115+0.000j is Ω = |(Sz - Λ)| = |(1.0 - 1)|
pair combination [36, 48] with c=-0.115+0.000j is Ω = |(Sz + Λ)| = |(-1.0 + 1)|
pair combination [26, 38] with c=0.102+0.000j is Ω = |(Sz - Λ)| = |(1.0 - 1)|
pair combination [34, 46] with c=0.102+0.000j is Ω = |(Sz + Λ)| = |(-1.0 + 1)|
pair combination [29, 41] with c=-0.057-0.000j is Ω = |(Sz - Λ)| = |(1.0 - 1)|
pair combination [37, 49] with c=-0.057+0.000j is Ω = |(Sz + Λ)| = |(-1.0 + 1)|
non-pair component 1 (Λ=0) is Ω = 0.0
pair combination [69, 89] with c=0.018-0.000j is Ω = |(Sz + Λ)| = |(-2.0 + 2)|
pair combination [65, 85] with c=0.018+0.000j is Ω = |(Sz - Λ)| = |(2.0 - 2)|
percent: {0.0: 100.0}
--- iso = 1 with Erel = 855.31 cm-1 ---


,Nr,State,S,Sz,Ecm,E,i_ci,ci_lbl,Lz,irrep,saverec,Ω,uOm,ipair,Term,type,pct,coeff
54,55,1.4,1.0,-1.0,0.00,-193.997608,26,1.4,0,4,6401.2,{-1.0},{1.0},54,(1)3Σ-,1,41.683382,0.646+0.000j
50,51,1.4,1.0,1.0,0.00,-193.997608,26,1.4,0,4,6401.2,{1.0},{1.0},50,(1)3Σ-,1,41.683382,0.646+0.000j
30,31,1.2,1.0,0.0,7036.86,-193.965546,18,1.2,1,2,6201.2,"{1.0, -1.0}",{1.0},42,(1)3Π,cos,7.392505,0.000-0.272j
7,8,1.3,0.0,0.0,8373.62,-193.959455,7,1.3,1,3,5301.2,"{1.0, -1.0}",{1.0},3,(1)1Π,sin,6.308387,-0.251-0.000j
32,33,3.2,1.0,0.0,13784.89,-193.934800,20,3.2,1,2,6201.2,"{1.0, -1.0}",{1.0},44,(2)3Π,cos,1.042818,-0.000+0.102j
10,11,4.3,0.0,0.0,24862.74,-193.884325,10,4.3,1,3,5301.2,"{1.0, -1.0}",{1.0},6,(3)1Π,sin,0.418204,0.065+0.000j
33,34,4.2,1.0,0.0,19724.30,-193.907738,21,4.2,1,2,6201.2,"{1.0, -1.0}",{1.0},45,(3)3Π,cos,0.378169,-0.000+0.061j
8,9,2.3,0.0,0.0,16248.13,-193.923576,8,2.3,1,3,5301.2,"{1.0, -1.0}",{1.0},4,(2)1Π,cos,0.305003,0.055+0.000j
56,57,1.4,1.0,1.0,22638.56,-193.894460,28,2.4,2,4,6402.2,"{3.0, -1.0}","{1.0, 3.0}",17,(1)3Δ,cos,0.118817,0.034+0.000j
62,63,1.4,1.0,-1.0,22638.56,-193.894460,28,2.4,2,4,6402.2,"{1.0, -3.0}","{1.0, 3.0}",23,(1)3Δ,cos,0.118817,0.034+0.000j


non-pair component 54 (Λ=0) is Ω = 1.0
non-pair component 50 (Λ=0) is Ω = 1.0
** component with ibas=30 (7.4%; Sz=0.0, Λ=1) lacks counterpart with ibas=42
    but unique Ω = 1.0
** component with ibas=7 (6.3%; Sz=0.0, Λ=1) lacks counterpart with ibas=3
    but unique Ω = 1.0
** component with ibas=32 (1.0%; Sz=0.0, Λ=1) lacks counterpart with ibas=44
    but unique Ω = 1.0
** component with ibas=10 (0.4%; Sz=0.0, Λ=1) lacks counterpart with ibas=6
    but unique Ω = 1.0
** component with ibas=33 (0.4%; Sz=0.0, Λ=1) lacks counterpart with ibas=45
    but unique Ω = 1.0
** component with ibas=8 (0.3%; Sz=0.0, Λ=1) lacks counterpart with ibas=4
    but unique Ω = 1.0
pair combination [17, 56] with c=0.034+0.000j is Ω = |(Sz - Λ)| = |(1.0 - 2)|
pair combination [23, 62] with c=0.034+0.000j is Ω = |(Sz + Λ)| = |(-1.0 + 2)|
non-pair component 83 (Λ=0) is Ω = 1.0
non-pair component 81 (Λ=0) is Ω = 1.0
non-pair component 55 (Λ=0) is Ω = 1.0
non-pair component 51 (Λ=0) is Ω = 1.0
percent: {1.0:

...

pair combination [73, 78] with c=0.480-0.000j is Ω = |(Sz + Λ)| = |(-1.0 + 3)|
pair combination [71, 76] with c=-0.480-0.000j is Ω = |(Sz - Λ)| = |(1.0 - 3)|
** component with ibas=59 (4.9%; Sz=0.0, Λ=2) lacks counterpart with ibas=20
    but unique Ω = 2.0
** component with ibas=67 (1.7%; Sz=0.0, Λ=2) lacks counterpart with ibas=87
    but unique Ω = 2.0
** component with ibas=60 (0.5%; Sz=0.0, Λ=2) lacks counterpart with ibas=21
    but unique Ω = 2.0
** component with ibas=61 (0.5%; Sz=0.0, Λ=2) lacks counterpart with ibas=22
    but unique Ω = 2.0
pair combination [29, 41] with c=0.020-0.000j is Ω = |(Sz + Λ)| = |(1.0 + 1)|
pair combination [37, 49] with c=0.020+0.000j is Ω = |(Sz - Λ)| = |(-1.0 - 1)|
pair combination [26, 38] with c=-0.014+0.000j is Ω = |(Sz + Λ)| = |(1.0 + 1)|
pair combination [34, 46] with c=-0.014-0.000j is Ω = |(Sz - Λ)| = |(-1.0 - 1)|
percent: {2.0: 99.9}
--- iso = 88 with Erel = 41639.29 cm-1 ---


,Nr,State,S,Sz,Ecm,E,i_ci,ci_lbl,Lz,irrep,saverec,Ω,uOm,ipair,Term,type,pct,coeff
74,75,1.2,2.0,-2.0,32244.83,-193.850690,32,1.2,3,2,7201.2,"{1.0, -5.0}","{1.0, 5.0}",79,5Φ,cos,21.945940,-0.000-0.468j
70,71,1.2,2.0,2.0,32244.83,-193.850690,32,1.2,3,2,7201.2,"{5.0, -1.0}","{1.0, 5.0}",75,5Φ,cos,21.945939,0.000+0.468j
79,80,1.3,2.0,-2.0,32244.83,-193.850690,33,1.3,3,3,7301.2,"{1.0, -5.0}","{1.0, 5.0}",74,5Φ,sin,21.945927,0.468-0.000j
75,76,1.3,2.0,2.0,32244.83,-193.850690,33,1.3,3,3,7301.2,"{5.0, -1.0}","{1.0, 5.0}",70,5Φ,sin,21.945927,0.468-0.000j
62,63,1.4,1.0,-1.0,22638.56,-193.894460,28,2.4,2,4,6402.2,"{1.0, -3.0}","{1.0, 3.0}",23,(1)3Δ,cos,1.858068,0.136-0.000j
56,57,1.4,1.0,1.0,22638.56,-193.894460,28,2.4,2,4,6402.2,"{3.0, -1.0}","{1.0, 3.0}",17,(1)3Δ,cos,1.858068,0.136-0.000j
23,24,1.1,1.0,-1.0,22638.65,-193.894459,15,1.1,2,1,6102.2,"{1.0, -3.0}","{1.0, 3.0}",62,(1)3Δ,sin,1.856933,0.000+0.136j
17,18,1.1,1.0,1.0,22638.65,-193.894459,15,1.1,2,1,6102.2,"{3.0, -1.0}","{1.0, 3.0}",56,(1)3Δ,sin,1.856933,-0.000-0.136j
63,64,2.4,1.0,-1.0,25689.70,-193.880558,29,4.4,2,4,6402.2,"{1.0, -3.0}","{1.0, 3.0}",24,(2)3Δ,cos,0.480494,-0.069+0.000j
57,58,2.4,1.0,1.0,25689.70,-193.880558,29,4.4,2,4,6402.2,"{3.0, -1.0}","{1.0, 3.0}",18,(2)3Δ,cos,0.480494,-0.069+0.000j


pair combination [74, 79] with c=-0.000-0.468j is Ω = |(Sz + Λ)| = |(-2.0 + 3)|
pair combination [70, 75] with c=0.000+0.468j is Ω = |(Sz - Λ)| = |(2.0 - 3)|
pair combination [23, 62] with c=0.136-0.000j is Ω = |(Sz + Λ)| = |(-1.0 + 2)|
pair combination [17, 56] with c=0.136-0.000j is Ω = |(Sz - Λ)| = |(1.0 - 2)|
pair combination [24, 63] with c=-0.069+0.000j is Ω = |(Sz + Λ)| = |(-1.0 + 2)|
pair combination [18, 57] with c=-0.069+0.000j is Ω = |(Sz - Λ)| = |(1.0 - 2)|
** component with ibas=10 (0.3%; Sz=0.0, Λ=1) lacks counterpart with ibas=6
    but unique Ω = 1.0
pair combination [25, 64] with c=0.000+0.053j is Ω = |(Sz + Λ)| = |(-1.0 + 2)|
pair combination [19, 58] with c=-0.000-0.053j is Ω = |(Sz - Λ)| = |(1.0 - 2)|
pair combination [68, 88] with c=-0.000-0.051j is Ω = |(Sz + Λ)| = |(-1.0 + 2)|
pair combination [66, 86] with c=-0.000-0.051j is Ω = |(Sz - Λ)| = |(1.0 - 2)|
** component with ibas=33 (0.1%; Sz=0.0, Λ=1) lacks counterpart with ibas=45
    but unique Ω = 1.0
** compone

,Nr,State,S,Sz,Ecm,E,i_ci,ci_lbl,Lz,irrep,saverec,Ω,uOm,ipair,Term,type,pct,coeff
75,76,1.3,2.0,2.0,32244.83,-193.850690,33,1.3,3,3,7301.2,"{5.0, -1.0}","{1.0, 5.0}",70,5Φ,sin,21.945933,0.000-0.468j
79,80,1.3,2.0,-2.0,32244.83,-193.850690,33,1.3,3,3,7301.2,"{1.0, -5.0}","{1.0, 5.0}",74,5Φ,sin,21.945933,-0.000+0.468j
70,71,1.2,2.0,2.0,32244.83,-193.850690,32,1.2,3,2,7201.2,"{5.0, -1.0}","{1.0, 5.0}",75,5Φ,cos,21.945926,0.468+0.000j
74,75,1.2,2.0,-2.0,32244.83,-193.850690,32,1.2,3,2,7201.2,"{1.0, -5.0}","{1.0, 5.0}",79,5Φ,cos,21.945926,0.468+0.000j
56,57,1.4,1.0,1.0,22638.56,-193.894460,28,2.4,2,4,6402.2,"{3.0, -1.0}","{1.0, 3.0}",17,(1)3Δ,cos,1.858085,0.000-0.136j
62,63,1.4,1.0,-1.0,22638.56,-193.894460,28,2.4,2,4,6402.2,"{1.0, -3.0}","{1.0, 3.0}",23,(1)3Δ,cos,1.858085,-0.000+0.136j
17,18,1.1,1.0,1.0,22638.65,-193.894459,15,1.1,2,1,6102.2,"{3.0, -1.0}","{1.0, 3.0}",56,(1)3Δ,sin,1.856918,-0.136-0.000j
23,24,1.1,1.0,-1.0,22638.65,-193.894459,15,1.1,2,1,6102.2,"{1.0, -3.0}","{1.0, 3.0}",62,(1)3Δ,sin,1.856918,-0.136-0.000j
57,58,2.4,1.0,1.0,25689.70,-193.880558,29,4.4,2,4,6402.2,"{3.0, -1.0}","{1.0, 3.0}",18,(2)3Δ,cos,0.480501,-0.000+0.069j
63,64,2.4,1.0,-1.0,25689.70,-193.880558,29,4.4,2,4,6402.2,"{1.0, -3.0}","{1.0, 3.0}",24,(2)3Δ,cos,0.480501,0.000-0.069j


pair combination [70, 75] with c=0.468+0.000j is Ω = |(Sz - Λ)| = |(2.0 - 3)|
pair combination [74, 79] with c=0.468+0.000j is Ω = |(Sz + Λ)| = |(-2.0 + 3)|
pair combination [17, 56] with c=0.000-0.136j is Ω = |(Sz - Λ)| = |(1.0 - 2)|
pair combination [23, 62] with c=-0.000+0.136j is Ω = |(Sz + Λ)| = |(-1.0 + 2)|
pair combination [18, 57] with c=-0.000+0.069j is Ω = |(Sz - Λ)| = |(1.0 - 2)|
pair combination [24, 63] with c=0.000-0.069j is Ω = |(Sz + Λ)| = |(-1.0 + 2)|
** component with ibas=6 (0.3%; Sz=0.0, Λ=1) lacks counterpart with ibas=10
    but unique Ω = 1.0
pair combination [19, 58] with c=-0.053-0.000j is Ω = |(Sz - Λ)| = |(1.0 - 2)|
pair combination [25, 64] with c=-0.053-0.000j is Ω = |(Sz + Λ)| = |(-1.0 + 2)|
pair combination [66, 86] with c=-0.051-0.000j is Ω = |(Sz - Λ)| = |(1.0 - 2)|
pair combination [68, 88] with c=0.051+0.000j is Ω = |(Sz + Λ)| = |(-1.0 + 2)|
** component with ibas=45 (0.1%; Sz=0.0, Λ=1) lacks counterpart with ibas=33
    but unique Ω = 1.0
** componen

In [827]:
# Construct matrix of transition dipoles
dmtot, dmabs = pto.tran_dip_mat(so_propmats, dimen)
dipZ = np.array([np.real(dmtot[i][i][2]) for i in range(dimen)])
df_soE['dipZ'] = dipZ

### Trig method

In [828]:
# Assign levels using the trig analysis alone
print(f'Applying trig method with used_LZ = {used_LZ} and use_recalculated = {use_recalculated}')
assmeth = 'trig'
assigned, om_left, iunass = pto.assign_Omega_trig(omsublev, dimen, omcompl, df_soE,
                                                 fsoci, Rstr)

Applying trig method with used_LZ = True and use_recalculated = True
Ω counts needed for input ΛS terms: {0: 16, 1: 30, 2: 22, 3: 14, 4: 6, 5: 2}
iso = 0 assigned Ω = 0.0
iso = 1 assigned Ω = 1.0
iso = 2 assigned Ω = 1.0
iso = 3 assigned Ω = 2.0
iso = 4 assigned Ω = 2.0
iso = 5 assigned Ω = 0.0
iso = 6 assigned Ω = 1.0
iso = 7 assigned Ω = 1.0
iso = 8 assigned Ω = 2.0
iso = 9 assigned Ω = 2.0
iso = 10 assigned Ω = 4.0
iso = 11 assigned Ω = 4.0
iso = 12 assigned Ω = 1.0
iso = 13 assigned Ω = 1.0
iso = 14 assigned Ω = 0.0
iso = 15 assigned Ω = 3.0
iso = 16 assigned Ω = 3.0
iso = 17 assigned Ω = 2.0
iso = 18 assigned Ω = 2.0
iso = 19 assigned Ω = 1.0
iso = 20 assigned Ω = 1.0
iso = 21 assigned Ω = 0.0
iso = 22 assigned Ω = 0.0
iso = 23 assigned Ω = 1.0
iso = 24 assigned Ω = 1.0
iso = 25 assigned Ω = 2.0
iso = 26 assigned Ω = 2.0
iso = 27 assigned Ω = 0.0
iso = 28 assigned Ω = 0.0
iso = 29 {3.0: 99.9, 1.0: 0.1}
iso = 30 {3.0: 99.9, 1.0: 0.1}
iso = 31 {1.0: 100.0, 3.0: 0.0}
iso = 32 {1.0: 1

In [829]:
nunk, iunass = pto.report_unassigned(assigned, om_left, omcompl)

There are 8 unassigned levels
Assignments remaining:
    Ω=0: 1
    Ω=1: 4
    Ω=3: 2
    Ω=4: 1
Unassigned sublevels:
iso = 29 {3.0: 99.9, 1.0: 0.1}
iso = 30 {3.0: 99.9, 1.0: 0.1}
iso = 31 {1.0: 100.0, 3.0: 0.0}
iso = 32 {1.0: 100.0, 3.0: 0.0}
iso = 36 {0.0: 99.2, '4.0?': 0.6}
iso = 38 {4.0: 99.5, 0.0: 0.5}
iso = 43 {1.0: 100.0, 3.0: 0.0}
iso = 44 {1.0: 100.0, 3.0: 0.0}
Maximum contamination is 0.80% for iso = 36


In [830]:
if nunk:
    # Maybe things are clean enough
    iunass = pto.confirm_questionable_omegas(iunass, assigned, omcompl, om_left, df_soE,
                                auto_accept = 10.)
    iunass = pto.report_unassigned(assigned, om_left, omcompl)
    nunk = len(iunass)

Contamination < 10.0% will be ignored
Contamination for level 29 = 0.1% automatically overlooked
Contamination for level 30 = 0.1% automatically overlooked
Contamination for level 31 = 0.0% automatically overlooked
Contamination for level 32 = 0.0% automatically overlooked
Contamination for level 36 = 0.8% automatically overlooked
Contamination for level 38 = 0.5% automatically overlooked
Contamination for level 43 = 0.0% automatically overlooked
Contamination for level 44 = 0.0% automatically overlooked
8 sublevels assigned
All levels assigned!
Maximum contamination is 0.80% for iso = 36


In [831]:
# Assign reflection parity for 0 levels, from trig analysis alone
pto.assign_0_parity_trig(df_soE, trig_rept, verbose=False)

In [832]:
# Use parity assignments and transition dipoles to complete parity assignments
pto.assign_some_0_parity_by_dipole(df_soE, dmabs)

In [833]:
pto.omega0_trandip_report(df_soE, dmabs)

There are 8 (0+) states and 8 (0-) states


,E,Eshift,Erel,Ω,dipZ,par
0,-194.010209,-2765.51,0.00,0.0,-1.041285,+
5,-193.970860,5870.64,8636.15,0.0,-1.115168,+
14,-193.963141,7564.70,10330.21,0.0,-1.134932,-
21,-193.947272,11047.56,13813.07,0.0,-1.086398,+
22,-193.937399,13214.53,15980.04,0.0,-0.255895,-
27,-193.928780,15106.03,17871.54,0.0,-0.725560,-
28,-193.919233,17201.47,19966.98,0.0,-0.672393,+
35,-193.914615,18215.00,20980.51,0.0,-0.408173,-
36,-193.911021,19003.75,21769.26,0.0,-0.532551,+
49,-193.901311,21134.87,23900.38,0.0,-0.268414,+


Largest cross transition dipole magnitude = 0.0


In [834]:
dfallow, dforbid = pto.trandip_assigned_report(df_soE, dmabs)

Finding maximum transition dipoles between assigned sublevels
Allowed:


,0+,0-,1,2,3,4,5
0+,1.751737,NaN,0.226072,NaN,NaN,NaN,NaN
0-,NaN,1.134932,0.156966,NaN,NaN,NaN,NaN
1,0.226072,0.156966,1.249717,0.165877,NaN,NaN,NaN
2,NaN,NaN,0.165877,1.078731,0.128497,NaN,NaN
3,NaN,NaN,NaN,0.128497,1.063278,0.123559,NaN
4,NaN,NaN,NaN,NaN,0.123559,1.056669,0.109477
5,NaN,NaN,NaN,NaN,NaN,0.109477,0.483176


Forbidden:


,0+,0-,1,2,3,4,5
0+,NaN,0.0,NaN,0.000118,0.006041,0.030795,0.007834
0-,0.0,NaN,NaN,0.00018,0.004454,0.002601,0.000474
1,NaN,NaN,NaN,NaN,0.004464,0.010427,0.000203
2,0.000118,0.00018,NaN,NaN,NaN,0.00009,0.000135
3,0.006041,0.004454,0.004464,NaN,NaN,NaN,0.00069
4,0.030795,0.002601,0.010427,0.00009,NaN,NaN,NaN
5,0.007834,0.000474,0.000203,0.000135,0.00069,NaN,NaN


Largest forbidden transition dipole = 0.0308


In [835]:
# Combine degenerate pairs of levels
tolcm = 21.  # energy degeneracy tolerance, cm-1
toldip = 0.01  # dipole moment / a.u.
tolau = tolcm / chem.AU2CM
cols = ['E', 'Eshift', 'Erel', 'dipZ', 'Ω', 'par']
thrl = [tolau, tolcm, tolcm,    toldip, 0,   0]
dflevels = m2.pair_lambdas(df_soE, cols=cols, thr=thrl)
# check that all necessary pairs have been made
for om in sorted(set(dflevels[chem.OMEGA])):
    if om == 0:
        continue
    sub1 = df_soE[df_soE[chem.OMEGA] == om]
    sub2 = dflevels[dflevels[chem.OMEGA] == om]
    n = len(sub2) - len(sub1) // 2
    if n:
        print(f'** {n} pairs are missing for {chem.OMEGA} = {om}')
        print(' Try increasing the thresholds in the call to m2.pair_lambdas()')
        ilone = []
        for i in dflevels[dflevels[chem.OMEGA] == om].index:
            if len(dflevels.at[i, 'index']) < 2:
                ilone.append(i)
        subdf = dflevels.loc[ilone]
        display(subdf)
        for i in range(0, len(subdf), 2):
            x1 = [subdf.iloc[i][col] for col in cols]
            x2 = [subdf.iloc[i+1][col] for col in cols]
            for j, col in enumerate(cols):
                try:
                    spr = np.round(abs(x2[j] - x1[j]), 3)
                except TypeError:
                    spr = x2[j] == x1[j]
                #print(f'{col:<8s}  {spr}   \t{thrl[j]}')
            print()
# Sort levels by energy
dflevels = dflevels.sort_values('E')
# Compute term composition of levels, from term composition of sublevels
dflevcompos = pto.level_compos_from_sublevel_compos(dflevels, dfcompos)
# Add rounded term compositions and leading term label to dflevels
pto.term_comp_approx(dflevels, dflevcompos)

Largest accepted energy non-degeneracy = 2.9 cm-1 for [33 34]


In [836]:
for om in sorted(set(dflevels.Ω.values)):
    omdf = dflevels[dflevels[chem.OMEGA] == om]
    fmt = {k: '{:.1f}' for k in ['Eshift', 'Erel', 'Ω']}
    fmt.update({'E': '{:.6f}'})
    print(f'Levels with Ω = {om}:')
    # don't display the Omega column
    cols = list(omdf.columns)
    cols.remove('Ω')
    if om == 0:
        display(omdf[cols].style.format(fmt).hide(axis='index'))
    else:
        # drop the parity column for Ω > 0
        cols.remove('par')
        display(omdf[cols].style.format(fmt).hide(axis='index'))

Levels with Ω = 0.0:


index,E,Eshift,Erel,dipZ,par,Lead,TC_approx
[0],-194.010209,-2765.5,0.0,-1.041285,+,(1)3Σ-,"{'(1)3Σ-': 80.1, '(1)1Σ+': 8.9, '(2)3Π': 5.3, '(1)3Π': 4.2, '(3)3Π': 1.3, '5Δ': 0.1}"
[5],-193.970860,5870.6,8636.1,-1.115168,+,(2)3Π,"{'(2)3Π': 51.2, '(1)1Σ+': 18.5, '(3)3Π': 16.6, '5Δ': 10.6, '(2)1Σ+': 1.2, '(1)3Σ-': 0.9, '(2)3Σ-': 0.6, '(1)3Π': 0.5}"
[14],-193.963141,7564.7,10330.2,-1.134932,-,(1)3Π,"{'(1)3Π': 93.6, '5Σ-': 4.7, '(2)1Σ-': 1.0, '3Σ+': 0.4, '(1)1Σ-': 0.1}"
[21],-193.947272,11047.6,13813.1,-1.086398,+,(1)1Σ-,"{'(1)1Σ-': 60.3, '3Σ+': 17.2, '(2)1Σ-': 9.2, '(3)3Π': 8.3, '5Δ': 4.1, '(2)3Π': 0.8, '5Σ-': 0.1}"
[22],-193.937399,13214.5,15980.0,-0.255895,-,3Σ+,"{'3Σ+': 43.8, '(1)1Σ-': 31.7, '(2)1Σ-': 14.7, '5Δ': 7.8, '5Σ-': 1.4, '(1)3Π': 0.3, '(2)3Π': 0.3}"
[27],-193.928780,15106.0,17871.5,-0.725560,-,(1)3Σ-,"{'(1)3Σ-': 83.4, '(1)3Π': 7.4, '(1)1Π': 6.3, '(2)3Π': 1.0, '(1)3Δ': 0.5, '(3)1Π': 0.4, '(3)3Π': 0.4, '(2)1Π': 0.3, '5Σ-': 0.2}"
[28],-193.919233,17201.5,19967.0,-0.672393,+,(1)3Π,"{'(1)3Π': 69.8, '(1)3Σ-': 11.4, '(1)1Π': 8.5, '5Σ-': 4.2, '(1)3Δ': 3.0, '(2)1Π': 1.4, '(2)3Π': 0.8, '(2)3Δ': 0.4, '(2)3Σ-': 0.3}"
[35],-193.914615,18215.0,20980.5,-0.408173,-,(3)3Π,"{'(3)3Π': 34.3, '5Σ-': 22.5, '(2)1Π': 9.4, '(2)3Σ-': 8.6, '(1)3Δ': 8.2, '(2)3Π': 4.8, '5Δ': 3.9, '(3)1Π': 3.5, '3Σ+': 2.4, '(1)3Σ-': 0.9, '(1)3Π': 0.9, '5Φ': 0.5}"
[36],-193.911021,19003.8,21769.3,-0.532551,+,(2)3Σ-,"{'(2)3Σ-': 32.1, '(3)1Π': 21.2, '(2)1Π': 15.6, '(1)3Δ': 11.8, '3Σ+': 6.7, '(2)3Π': 6.2, '(3)3Δ': 1.2, '5Δ': 1.1, '5Σ-': 0.9, '(1)3Σ-': 0.9, '5Φ': 0.7, '(1)1Π': 0.6, '(1)3Π': 0.4, '(2)3Δ': 0.4, '(3)3Π': 0.3}"
[49],-193.901311,21134.9,23900.4,-0.268414,+,(3)3Δ,"{'(3)3Δ': 87.5, '(3)1Π': 6.8, '(2)3Δ': 2.0, '3Σ+': 1.4, '5Φ': 1.0, '(1)1Π': 0.5, '(2)1Π': 0.5, '(1)3Π': 0.3}"


Levels with Ω = 1.0:


index,E,Eshift,Erel,dipZ,Lead,TC_approx
"[1, 2]",-194.006312,-1910.2,855.3,-1.074535,(1)3Π,"{'(1)3Π': 48.3, '(1)1Σ+': 42.0, '(2)3Π': 5.9, '(1)3Σ-': 1.7, '(3)3Π': 1.0, '(2)1Σ+': 0.9, '5Δ': 0.1}"
"[6, 7]",-193.969326,6207.3,8972.8,-1.109912,(3)3Π,"{'(3)3Π': 75.1, '(2)3Π': 23.8, '5Δ': 0.6, '(2)1Σ+': 0.3}"
"[12, 13]",-193.964513,7263.5,10029.1,-1.122220,5Δ,"{'5Δ': 87.7, '(2)3Π': 7.6, '(3)3Π': 4.0, '(1)1Σ+': 0.5, '(1)3Σ-': 0.2, '(2)1Σ+': 0.1}"
"[19, 20]",-193.948077,10870.9,13636.4,-1.037139,(3)3Π,"{'(3)3Π': 54.2, '5Σ-': 20.9, '(2)3Π': 12.4, '(1)1Σ-': 7.1, '3Σ+': 2.8, '(2)1Σ-': 1.3, '(1)3Π': 0.9, '5Δ': 0.3}"
"[23, 24]",-193.935040,13732.2,16497.7,-0.249849,5Δ,"{'5Δ': 79.2, '3Σ+': 6.9, '(2)3Π': 6.6, '(3)3Π': 5.3, '(2)1Σ-': 2.0}"
"[31, 32]",-193.915853,17943.2,20708.7,-0.287443,(2)3Π,"{'(2)3Π': 43.2, '(2)1Π': 32.9, '(1)1Π': 15.4, '(1)3Π': 3.2, '5Σ-': 1.1, '(1)3Σ-': 1.0, '(2)3Σ-': 1.0, '5Δ': 1.0, '(1)3Δ': 0.5, '(3)3Δ': 0.3, '(3)3Π': 0.2, '3Σ+': 0.1}"
"[39, 40]",-193.907502,19776.0,22541.5,-0.327784,(1)3Δ,"{'(1)3Δ': 36.4, '(2)3Σ-': 32.5, '(3)1Π': 12.7, '5Δ': 6.0, '(3)3Π': 5.7, '5Φ': 1.7, '(1)3Π': 1.3, '(2)3Δ': 1.3, '(2)1Π': 1.0, '5Σ-': 0.8, '(2)3Π': 0.3, '(3)3Δ': 0.3}"
"[43, 44]",-193.904593,20414.6,23180.1,-0.765555,(3)1Π,"{'(3)1Π': 22.6, '(1)3Δ': 21.7, '(3)3Π': 15.2, '5Δ': 11.1, '3Σ+': 9.9, '(2)3Σ-': 6.3, '5Φ': 5.7, '(2)3Δ': 2.7, '(1)1Π': 2.1, '(2)3Π': 1.0, '(3)3Δ': 0.8, '(1)3Σ-': 0.5, '(1)3Π': 0.3}"
"[51, 52]",-193.895572,22394.3,25159.8,-0.232878,1Δ,"{'1Δ': 46.5, '(1)3Π': 44.6, '5Σ-': 3.3, '3Φ': 2.3, '(3)3Π': 1.7, '(1)3Δ': 1.2, '(2)3Δ': 0.2, '(2)3Π': 0.1}"
"[60, 61]",-193.880364,25732.1,28497.6,-0.200425,(3)3Π,"{'(3)3Π': 42.4, '(1)3Δ': 41.6, '5Σ-': 9.8, '5Φ': 1.6, '5Δ': 1.4, '(2)3Δ': 1.3, '1Δ': 1.2, '3Φ': 0.5}"


Levels with Ω = 2.0:


...

index,E,Eshift,Erel,dipZ,Lead,TC_approx
"[15, 16]",-193.949581,10540.9,13306.4,-1.063277,5Σ-,"{'5Σ-': 47.8, '(3)3Π': 26.8, '(2)3Π': 12.4, '3Σ+': 7.4, '5Δ': 2.7, '(1)3Π': 2.4, '(1)1Σ-': 0.5}"
"[29, 30]",-193.918994,17253.8,20019.4,-0.982199,(1)1Π,"{'(1)1Π': 62.4, '(2)3Π': 17.3, '(1)3Π': 11.0, '(2)1Π': 4.9, '(2)3Σ-': 1.1, '(1)3Σ-': 0.9, '(1)3Δ': 0.6, '3Σ+': 0.5, '5Σ-': 0.4, '5Δ': 0.3, '(3)3Δ': 0.3, '(3)3Π': 0.2}"
"[45, 46]",-193.902874,20791.9,23557.4,-0.859181,3Σ+,"{'3Σ+': 68.2, '(3)1Π': 20.1, '(3)3Δ': 5.7, '(1)3Δ': 2.5, '5Σ-': 1.2, '(2)3Σ-': 0.9, '(1)3Π': 0.5, '(3)3Π': 0.3, '5Δ': 0.2, '5Φ': 0.2}"
"[47, 48]",-193.902281,20922.0,23687.5,-0.514545,(2)3Δ,"{'(2)3Δ': 87.8, '(3)3Π': 2.9, '(3)3Δ': 2.1, '5Δ': 1.9, '5Φ': 1.9, '(3)1Π': 1.5, '(1)3Π': 0.7, '(2)1Π': 0.4, '3Σ+': 0.3, '(1)1Π': 0.2, '(2)3Π': 0.1}"
"[55, 56]",-193.892832,22995.8,25761.3,-0.557882,(2)3Π,"{'(2)3Π': 95.8, '(3)3Π': 1.4, '5Σ-': 1.3, '5Δ': 1.1, '(1)3Π': 0.1, '(2)3Δ': 0.1, '3Φ': 0.1}"
"[67, 68]",-193.873907,27149.3,29914.8,-0.826314,(3)3Δ,"{'(3)3Δ': 94.5, '3Φ': 4.2, '(1)3Π': 0.3, '(3)3Π': 0.3, '5Δ': 0.3, '(1)3Δ': 0.2, '5Φ': 0.2}"
"[84, 85]",-193.848526,32719.8,35485.3,-0.478508,5Δ,"{'5Δ': 94.3, '3Φ': 4.1, '5Φ': 1.4, '(3)3Π': 0.2}"


Levels with Ω = 4.0:


index,E,Eshift,Erel,dipZ,Lead,TC_approx
"[10, 11]",-193.965732,6996.0,9761.5,-1.056668,(2)1Σ+,"{'(2)1Σ+': 90.9, '(2)3Σ-': 3.3, '(1)3Π': 3.2, '(2)3Π': 1.6, '5Δ': 0.7, '(1)3Σ-': 0.3}"
"[37, 38]",-193.910707,19072.7,21838.2,-0.168727,(2)1Π,"{'(2)1Π': 31.6, '(3)3Π': 22.1, '(2)3Π': 18.8, '(2)3Σ-': 10.3, '(1)3Δ': 5.8, '(1)1Π': 3.6, '(2)3Δ': 3.0, '5Σ-': 1.6, '(1)3Σ-': 1.0, '(3)1Π': 0.8, '5Δ': 0.7, '(1)3Π': 0.3, '(3)3Δ': 0.3, '5Φ': 0.1}"
"[77, 78]",-193.861689,29830.9,32596.4,-0.478577,(2)3Δ,"{'(2)3Δ': 52.4, '(1)3Δ': 40.2, '5Δ': 4.1, '(3)3Δ': 1.7, '5Φ': 1.2, '1Φ': 0.3}"


Levels with Ω = 5.0:


index,E,Eshift,Erel,dipZ,Lead,TC_approx
"[65, 66]",-193.874027,27122.9,29888.4,-0.483176,(2)3Δ,"{'(2)3Δ': 71.4, '(3)3Π': 15.3, '5Δ': 5.9, '(1)3Δ': 5.8, '5Σ-': 0.7, '1Δ': 0.3, '(3)3Δ': 0.2, '5Φ': 0.2, '(2)3Π': 0.1}"


In [837]:
# Get omega values as str
Omstr = []  # omega with parity
Tstr =  []  # (leading) term symbol including omega
for ilev, (trm, om, par) in enumerate(zip(dflevels.Lead, dflevels.Ω, dflevels.par)):
    ostr = chem.halves(om)
    if om == 0:
        ostr += par
    Omstr.append(ostr)
    tstr = trm + '_' + ostr
    Tstr.append(tstr)
dflevels['Olbl'] = chem.enumerative_prefix(Omstr)
dflevels['Tlbl'] = Tstr

In [838]:
# Correct any duplicates
dupls = Counter(Tstr)
for tstr, n in dupls.items():
    if n > 1:
        print(f'** {n} levels are assigned as {tstr}')
        subdf = dflevels[dflevels.Tlbl == tstr] # duplicates
        display(subdf)
        # find unused leading terms
        om = subdf.iloc[0]['Ω']
        allleads = []
        for trm in dftermci.Term:
            t = chem.strip_enumerative_prefix(trm)
            omposs = chem.omega_possible_from_term(t)
            if om in omposs:
                allleads.append(trm)
        omdf = dflevels[dflevels.Ω == om]
        omiss = set(allleads) - set(omdf.Lead)
        if len(omiss) > 0:
            print(f'No Ω={om} level has been assigned to terms {omiss}')
            for t in omiss:
                # does this term contribute significantly to any of these levels?
                signif = False
                for TC in subdf.TC_approx.values:
                    leadwt = max(TC.values())
                    twt = TC.get(t, 0)
                    if twt > leadwt - 15.:
                        # within 15% of leader
                        signif = True
                if signif:
                    iass = input(f'Level ({subdf.index.tolist()}) to assign Lead = {t}? (blank to skip) ')
                    if len(iass):
                        iass = int(iass)
                        dflevels.at[iass, 'Lead'] = t
                        tlbl = t + f'_{chem.halves(om)}'
                        if om == 0:
                            tlbl += dflevels.at[iass, 'par']
                        dflevels.at[iass, 'Tlbl'] = tlbl
        else:
            print(f'No unassigned leading terms for Ω={om}')

** 2 levels are assigned as (1)3Π_2


,index,E,Eshift,Erel,Ω,dipZ,par,Lead,TC_approx,Olbl,Tlbl
31,"[3, 4]",-193.988322,2038.170,4803.680,2.0,-1.064038,,(1)3Π,"{'(1)3Π': 43.6, '(1)1Σ+': 30.0, '(1)3Σ-': 16.9...",(1)2,(1)3Π_2
37,"[53, 54]",-193.893819,22779.035,25544.545,2.0,-0.176710,,(1)3Π,"{'(1)3Π': 44.4, '1Δ': 43.8, '5Σ-': 4.2, '3Φ': ...",(7)2,(1)3Π_2


No Ω=2.0 level has been assigned to terms {'(2)3Δ', '(3)3Δ', '1Δ', '(3)3Π', '3Φ', '(1)3Δ'}
Level ([31, 37]) to assign Lead = 1Δ? (blank to skip) 37
** 3 levels are assigned as (3)3Π_1


,index,E,Eshift,Erel,Ω,dipZ,par,Lead,TC_approx,Olbl,Tlbl
17,"[6, 7]",-193.969326,6207.305,8972.815,1.0,-1.109912,,(3)3Π,"{'(3)3Π': 75.1, '(2)3Π': 23.8, '5Δ': 0.6, '(2)...",(2)1,(3)3Π_1
19,"[19, 20]",-193.948077,10870.875,13636.390,1.0,-1.037139,,(3)3Π,"{'(3)3Π': 54.2, '5Σ-': 20.9, '(2)3Π': 12.4, '(...",(4)1,(3)3Π_1
25,"[60, 61]",-193.880364,25732.110,28497.620,1.0,-0.200425,,(3)3Π,"{'(3)3Π': 42.4, '(1)3Δ': 41.6, '5Σ-': 9.8, '5Φ...",(10)1,(3)3Π_1


No Ω=1.0 level has been assigned to terms {'(2)3Σ-', '(2)3Δ', '3Σ+', '5Σ-', '(1)1Π', '(1)3Σ-', '(2)1Π'}
** 2 levels are assigned as 5Δ_1


...

No Ω=0.0 level has been assigned to terms {'(2)1Σ-', '5Σ-', '(2)1Σ+', '5Δ', '(1)1Σ+'}
** 2 levels are assigned as 5Φ_1


,index,E,Eshift,Erel,Ω,dipZ,par,Lead,TC_approx,Olbl,Tlbl
26,"[69, 70]",-193.872202,27523.545,30289.055,1.0,-0.328046,,5Φ,"{'5Φ': 92.1, '(1)3Δ': 4.9, '5Δ': 1.7, '(2)3Δ':...",(11)1,5Φ_1
30,"[88, 89]",-193.820486,38873.780,41639.290,1.0,-0.520361,,5Φ,{'5Φ': 100.0},(15)1,5Φ_1


No Ω=1.0 level has been assigned to terms {'(2)3Σ-', '(2)3Δ', '3Σ+', '5Σ-', '(1)1Π', '(1)3Σ-', '(2)1Π'}
** 2 levels are assigned as 3Φ_1


,index,E,Eshift,Erel,Ω,dipZ,par,Lead,TC_approx,Olbl,Tlbl
27,"[71, 72]",-193.869721,28068.06,30833.57,1.0,-0.292139,,3Φ,"{'3Φ': 77.3, '1Φ': 18.5, '5Δ': 2.2, '(2)3Δ': 0...",(12)1,3Φ_1
29,"[82, 83]",-193.849130,32587.23,35352.74,1.0,-0.762833,,3Φ,"{'3Φ': 95.8, '5Δ': 4.1}",(14)1,3Φ_1


No Ω=1.0 level has been assigned to terms {'(2)3Σ-', '(2)3Δ', '3Σ+', '5Σ-', '(1)1Π', '(1)3Σ-', '(2)1Π'}


In [839]:
f = fdir.split(os.sep)[-3]
print(f'Final levels and assignments for R = {Rstr} Å ({f})')
dflevels['Tlbl'] = chem.enumerative_prefix(dflevels.Tlbl, style='alpha')
display(dflevels[['Tlbl', 'Olbl', 'E', 'Erel', 'Eshift']].style.format(fmt))

Final levels and assignments for R = 1.727 Å (14S_17T_5Q)


,Tlbl,Olbl,E,Erel,Eshift
0,(1)3Σ-_0+,(1)0+,-194.010209,0.0,-2765.5
16,(1)3Π_1,(1)1,-194.006312,855.3,-1910.2
31,(1)3Π_2,(1)2,-193.988322,4803.7,2038.2
1,(2)3Π_0+,(2)0+,-193.970860,8636.1,5870.6
17,a (3)3Π_1,(2)1,-193.969326,8972.8,6207.3
32,(2)3Σ-_2,(2)2,-193.968431,9169.1,6403.6
49,(2)1Σ+_4,(1)4,-193.965732,9761.5,6996.0
18,a 5Δ_1,(3)1,-193.964513,10029.1,7263.5
8,(1)3Π_0-,(1)0-,-193.963141,10330.2,7564.7
42,5Σ-_3,(1)3,-193.949581,13306.4,10540.9


In [840]:
# copy to clipboard
dflevels[['Tlbl', 'Olbl', 'E']].to_clipboard(index_label=f'R={Rstr}')
print(f'Levels for R={round(R, 5)} copied to clipboard, sorted by energy')

Levels for R=1.727 copied to clipboard, sorted by energy


In [841]:
# save to a file
#fpkl = f'R_{Rstr}'.replace('.', 'p') + '.pkl'
fpkl = froot.replace('.pro', '.pkl')
fpkl = os.sep.join([fdir, fpkl])
pkl = True
if os.path.isfile(fpkl):
    # verify overwriting
    q = input(f'Overwrite pickle file for R={round(R, 5)}? ')
    pkl = (q.lower()[0] == 'y')
if pkl:
    dflevels.to_pickle(fpkl)
    print(f'Table saved to file {fpkl}')  # read back using pd.read_pickle(fpkl)
else:
    print('Pickle file NOT written')

Overwrite pickle file for R=1.727? y
Table saved to file C:\Users\irikura\OneDrive - NIST\Karl\PtO\14S_17T_5Q\\\ac5z_r1p727_lz.pkl


In [842]:
# Transpose and copy to clipboard for Excel, sorted by Olbl
dfout = dflevels[['Olbl', 'Tlbl', 'E']].copy()
dfout = dfout.sort_values('Olbl')
rcol = f'R={Rstr}'
dfout.rename(columns={'Olbl': rcol}, inplace=True)
dfout.set_index(rcol, inplace=True)
display(dfout.T)
print('Levels copied to clipboard, sorted by Olbl')
dfout.T.to_clipboard(index_label=rcol)

R=1.727,(1)0+,(1)0-,(1)1,(1)2,(1)3,(1)4,(10)1,(10)2,(11)1,(11)2,...,(7)1,(7)2,(7)3,(8)0+,(8)0-,(8)1,(8)2,(9)1,(9)2,5
Tlbl,(1)3Σ-_0+,(1)3Π_0-,(1)3Π_1,(1)3Π_2,5Σ-_3,(2)1Σ+_4,c (3)3Π_1,c 5Δ_2,a 5Φ_1,5Φ_2,...,(1)3Δ_1,1Δ_2,5Δ_3,1Φ_0+,5Φ_0-,(3)1Π_1,b 5Σ-_2,1Δ_1,b 5Δ_2,(2)3Δ_5
E,-194.010209,-193.963141,-194.006312,-193.988322,-193.949581,-193.965732,-193.880364,-193.863318,-193.872202,-193.83476,...,-193.907502,-193.893819,-193.848526,-193.865867,-193.861099,-193.904593,-193.884325,-193.895572,-193.877409,-193.874027


Levels copied to clipboard, sorted by Olbl


In [ ]:
1/0

### Basis states deriving from a term

In [ ]:
selterm = '5Δ'
BS1term = pto.basis_states_from_term(selterm, dftermci, dfsobas)

### Term composition of a level

In [ ]:
levsel = '3Φ_4'
dfcomp1, (ilev, tlbl, olbl) = pto.term_compos_of_one_level(levsel, dflevels, dfcompos)
#np.round(dfcomp1, 3)

### Distribution of a term among levels

In [ ]:
termsel = '5Σ-'
dfdistrib = pto.term_distribution_among_levels(termsel, dflevcompos, dftermci, dflevels)
print(f'Total = {dfdistrib[termsel].sum():.2f} %')
cols = [termsel, 'index', 'E', 'Erel', 'Ω', 'dipZ', 'Olbl', 'Tlbl', 'TC_approx']
dfdistrib[cols].style.format({termsel: '{:.2f}'})